<a href="https://colab.research.google.com/github/AgarwalMayank2/Face_Detection/blob/KNN_Tavishi/KNN_Image_Identification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Loading dataset

In [3]:
import kagglehub

# Download latest version
path = kagglehub.dataset_download("jessicali9530/lfw-dataset")

100%|██████████| 112M/112M [00:01<00:00, 103MB/s]

Extracting files...


In [4]:
print(path)

/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4


In [5]:
import pandas as pd
df1=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/people.csv")
print(df1.head(10))

                 name  images
0  Abdel_Madi_Shabneh     1.0
1        Abdul_Rahman     1.0
2        Abel_Pacheco     4.0
3        Adriana_Lima     1.0
4         Afton_Smith     1.0
5        Ahmad_Jbarah     1.0
6      Akhmed_Zakayev     3.0
7     Alan_Dershowitz     1.0
8   Alanis_Morissette     1.0
9    Alberto_Gonzales     1.0


In [6]:
df2=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTrain.csv")
print(df2.head(10))

                  name  images
0              AJ_Cook       1
1        Aaron_Eckhart       1
2      Aaron_Patterson       1
3        Aaron_Peirsol       4
4           Aaron_Pena       1
5         Aaron_Sorkin       2
6     Abbas_Kiarostami       1
7  Abdel_Aziz_Al-Hakim       1
8   Abdel_Madi_Shabneh       1
9  Abdel_Nasser_Assidi       2


In [7]:
df3=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTest.csv")
print(df3.head(10))

                           name  images
0                      AJ_Lamas       1
1                   Aaron_Guiel       1
2                  Aaron_Tippin       1
3                     Abba_Eban       1
4        Abdul_Majeed_Shobokshi       1
5             Abdulaziz_Kamilov       1
6                  Abdullah_Gul      19
7              Abdullatif_Sener       2
8                  Abel_Aguilar       1
9  Abid_Hamid_Mahmud_Al-Tikriti       3


In [8]:
import os
images_address="/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled"
labels=os.listdir(images_address)
paths_of_images=[]
for root, dirs, files in os.walk(images_address):
    for file in files:
        paths_of_images.append(os.path.join(root, file))

print(paths_of_images[:5])

['/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Wesley_Clark/Wesley_Clark_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Wesley_Clark/Wesley_Clark_0002.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Myung_Yang/Myung_Yang_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Ronald_Brower/Ronald_Brower_0001.jpg', '/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/lfw-deepfunneled/lfw-deepfunneled/Lance_Armstrong/Lance_Armstrong_0006.jpg']


# Extracting features using CNN

In [17]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [18]:
print(device)

cuda


In [9]:
import torch
import torch.nn as nn
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image

# Load pre-trained ResNet-50 model
resnet = models.resnet50(pretrained=True)
# Remove the last fully connected layer
resnet = nn.Sequential(*list(resnet.children())[:-1])
# Set the model to evaluation mode
resnet.eval()

# Define a function to extract features from an image
def extract_features(image_path, model):
    # Load and preprocess the image
    image = Image.open(image_path).convert('RGB')
    preprocess = transforms.Compose([
        transforms.Resize(256),
        transforms.CenterCrop(224),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
    ])
    image = preprocess(image)
    # Add batch dimension
    image = image.unsqueeze(0)
    # Extract features
    with torch.no_grad():
        features = model(image)
    # Remove the batch dimension
    features = features.squeeze(0)
    return features

# Example usage
# image_path = 'example_image.png'  # Replace 'example_image.jpg' with your image path
# features = extract_features(image_path, resnet)
# print(features.shape)  # Output: torch.Size([2048])
# print(features)

# Save the extracted features
torch.save(features, 'extracted_features.pt')  # Save the features to a file

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 111MB/s]


# Extracting features using LBP

In [26]:
#This code can be used to extract Histogram of Gradient (HoG) Features.
#It takes Image Path and returns HoG feature.
import cv2
import numpy as np
from matplotlib import pyplot as plt

def get_pixel(img, center, x, y):
    new_value = 0
    try:
        if img[x][y] >= center:
            new_value = 1
    except:
        pass
    return new_value

def lbp_calculated_pixel(img, x, y):
    center = img[x][y]
    val_ar = []
    val_ar.append(get_pixel(img, center, x-1, y+1))     # top_right
    val_ar.append(get_pixel(img, center, x, y+1))       # right
    val_ar.append(get_pixel(img, center, x+1, y+1))     # bottom_right
    val_ar.append(get_pixel(img, center, x+1, y))       # bottom
    val_ar.append(get_pixel(img, center, x+1, y-1))     # bottom_left
    val_ar.append(get_pixel(img, center, x, y-1))       # left
    val_ar.append(get_pixel(img, center, x-1, y-1))     # top_left
    val_ar.append(get_pixel(img, center, x-1, y))       # top

    power_val = [1, 2, 4, 8, 16, 32, 64, 128]
    val = 0
    for i in range(len(val_ar)):
        val += val_ar[i] * power_val[i]
    return val


def calcLBP(img):
    device="cuda" if torch.cuda.is_available() else "cpu"

    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    img_tensor = torch.tensor(img_gray, dtype=torch.float32, device=device)

    height, width = img_tensor.shape


    img_lbp = torch.zeros((height, width), dtype=torch.float32, device=device)
    for i in range(0, height):
        for j in range(0, width):
             img_lbp[i, j] = lbp_calculated_pixel(img_tensor, i, j)
    # hist_lbp = cv2.calcHist([img_lbp], [0], None, [256], [0, 256])
    hist_lbp=torch.histc(img_lbp, bins=256, min=0, max=255)
    hist_lbp /= hist_lbp.sum()
    return(hist_lbp)


# image_file = 'example_img.png'
# img = cv2.imread(image_file)
# lbpFeature=calcLBP(img)
# print(lbpFeature)

# Extracting features using HoG

In [24]:
#This code can be used to extract Histogram of Gradient (HoG) Features.
#It takes Image Path and returns HoG feature.
from skimage.io import imread
from skimage.transform import resize
from skimage.feature import hog
import matplotlib.pyplot as plt
import cupy as cp


def compute_hog(img):

  device="cuda" if torch.cuda.is_available() else "cpu"

  img_gpu=cp.asarray(img)
  #resizing image
  img_gpu_resized = cp.asarray(resize(cp.asnumpy(img_gpu), (128 * 4, 64 * 4)))  # Skimage resize is CPU-only
  # resized_img = resize(img, (128*4, 64*4))
  #creating hog features
  # fd, hog_image = hog(resized_img, orientations=9, pixels_per_cell=(8, 8),
  #                   cells_per_block=(2, 2), visualize=True, channel_axis=-1)

  fd, hog_image = hog(cp.asnumpy(img_gpu_resized), orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, channel_axis=-1)

  fd_tensor = torch.tensor(fd, dtype=torch.float32, device=device)
  hog_image_tensor = torch.tensor(hog_image, dtype=torch.float32, device=device)

  return fd_tensor, hog_image_tensor

#reading the image
# img = imread('example_image.png')
# Hog_feature, hog_image=compute_hog(img)
# print(Hog_feature.shape)

# Combining all features

In [27]:
combined_features=[]
for path in paths_of_images:
  img=cv2.imread(path)
  cnn_features=extract_features(path, resnet)
  lbp_features=calcLBP(img)
  # lbp_features=torch.from_numpy(lbp_features).to(device) #Convert to tensor and move to device
  Hog_feature, hog_image=compute_hog(img)
  # Hog_feature = torch.from_numpy(Hog_feature).to(device) #Convert to tensor and move to device

  cnn_features = cnn_features.cpu().detach().numpy().flatten()
  lbp_features = lbp_features.reshape(-1).cpu().detach().numpy()
  Hog_feature = Hog_feature.cpu().detach().numpy()

  combined_features.append(np.concatenate((cnn_features, lbp_features, Hog_feature)))

KeyboardInterrupt: 

In [28]:
import cv2
import torch
import numpy as np
import concurrent.futures
from torchvision import models, transforms
from skimage.feature import hog
from skimage.transform import resize

# Load model on GPU
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
resnet = models.resnet18(pretrained=True).to(device)
resnet.eval()

# Image preprocessing for ResNet
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
])

# Feature extraction function
def extract_features(path, model):
    img = cv2.imread(path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)  # Convert to RGB
    img = transform(img).unsqueeze(0).to(device)  # Convert to tensor & move to GPU

    with torch.no_grad():
        features = model(img)

    return features.cpu().detach().numpy().flatten()  # Move to CPU & flatten

# GPU-accelerated LBP function
def calcLBP(img):
    img_gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img_tensor = torch.tensor(img_gray, dtype=torch.float32).to(device)

    lbp_features = torch.zeros_like(img_tensor)
    shifts = [(-1, 1), (0, 1), (1, 1), (1, 0), (1, -1), (0, -1), (-1, -1), (-1, 0)]

    for i, (dx, dy) in enumerate(shifts):
        shifted = torch.roll(img_tensor, shifts=(dx, dy), dims=(0, 1))
        lbp_features += (shifted >= img_tensor) * (2 ** i)

    hist_lbp = torch.histc(lbp_features, bins=256, min=0, max=256).to(device)
    return hist_lbp

# GPU-accelerated HOG
def compute_hog(img):
    img = resize(img, (128 * 4, 64 * 4))  # Resize
    img_tensor = torch.tensor(img, dtype=torch.float32).to(device)

    fd, hog_image = hog(img_tensor.cpu().numpy(), orientations=9, pixels_per_cell=(8, 8),
                        cells_per_block=(2, 2), visualize=True, channel_axis=-1)

    return torch.tensor(fd, dtype=torch.float32).to(device), hog_image

# Parallel Processing
def process_image(path):
    img = cv2.imread(path)

    cnn_features = extract_features(path, resnet)
    lbp_features = calcLBP(img)
    hog_features, _ = compute_hog(img)

    lbp_features = lbp_features.cpu().detach().numpy().flatten()
    hog_features = hog_features.cpu().detach().numpy().flatten()

    return np.concatenate((cnn_features, lbp_features, hog_features))

# Run in parallel
combined_features = []
with concurrent.futures.ThreadPoolExecutor(max_workers=4) as executor:
    results = list(executor.map(process_image, paths_of_images))

combined_features = np.array(results)

/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.11/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to /root/.cache/torch/hub/checkpoints/resnet18-f37072fd.pth
100%|██████████| 44.7M/44.7M [00:00<00:00, 153MB/s]


# Feature Extraction final

In [ ]:
df=pd.DataFrame(combined_features)
print(df.head())

# Applying PCA for dimensionality reduction

In [ ]:
from sklearn.decomposition import PCA

pca=PCA(n_components=50)

final_features=pca.fit_transform(combined_features)

print(final_features.shape)

# Train and Test dataframes

In [ ]:
train_df=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTrain.csv")
test_df=pd.read_csv("/root/.cache/kagglehub/datasets/jessicali9530/lfw-dataset/versions/4/peopleDevTest.csv")

train_names=train_df['name'].tolist()
test_names=test_df['name'].tolist()

train_indices=[i for i,name in enumerate(labels) if name in train_names]
test_indices=[j for j,name in enumerate(labels) if name in test_names]

X_train, X_test=final_features[train_indices], final_features[test_indices]
y_train, y_test=labels[train_indices], labels[test_indices]

print("Train size = ", X_train.shape)
print("Test size = ", X_test.shape)
print("Labels size(Train dataset)= ", y_train.shape)
print("Labels size(Test dataset) = ", y_test.shape)

# KNN Model

In [ ]:
def euclidean_dist(x,y):
  return np.sqrt(np.sum(x-y)**2)

In [ ]:
k=3

In [ ]:
predictions=[]
for i in range(X_test.shape[0]):
  distances=[]
  nearest_labels=[]
  for j in range(X_train.shape[0]):
    # append the pair (distance, index) to distances
    distances.append(euclidean_dist(X_test[i], X_train[j]), j)
  # sorting distances on the basis of first item of pair i.e. distance
  distances.sort(key=lambda x:x[0])
  for k in range(k):
    nearest_labels.append(y_train[distances[k][1]])

  # finding label with maximum frequency
  max_freq_index=0
  for label in nearest_labels:
    freq=nearest_labels.count(label)
    if freq>nearest_labels.count(nearest_labels[max_freq_index]):
      max_freq_index=nearest_labels.index(label)

  predictions.append(nearest_labels[max_freq_index])
  print(X_test[i], "\t", nearest_labels[max_freq_index])

print(predictions)